In [1]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install jupysql
# pip install notebook

In [3]:
# If on windows, this only works if you're running Jupyter as an admin
pip install duckdb-engine

  Using cached duckdb_engine-0.13.1-py3-none-any.whl (47 kB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
import sqlalchemy
from datetime import date
import json
%load_ext sql

Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
# Set up SQL connection
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
current_date = date.today().strftime('%Y-%m-%d')

In [4]:
# Replace with your filepath. Forward slashes work for Windows too
%sql duckdb:///C:/Users/Admin/billboard/raw_db.duckdb

In [5]:
%sql USE raw_db;
%sql USE raw_db.billboard;

,Success


In [20]:
%sql SELECT * FROM raw_db.billboard.artist LIMIT 2;

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
1,2,Morgan Wallen,0.0,2024-08-23


In [7]:
# Update this for this week's rankings
rankings = [
    'A Bar Song (Tipsy)'
    , 'I Had Some Help'
    , 'Pour Me A Drink'
    , 'Cowgirls'
    , "Ain't No Love In Oklahoma"
    , 'Pink Skies'
    , 'Guy For That'
    , 'Lies Lies Lies'
    , 'Miles On It'
    , 'Austin'
    , 'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'I Am Not Okay'
    , 'California Sober' # Post Malone feat Chris Stapleton
    , '28'
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'High Road'
    , 'Dirt Cheap'
    , 'Yours' # Post Malone
    , 'Have The Heart'
    , 'Bulletproof'
    , 'Whiskey Whiskey'
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , 'You Look Like You Love Me'
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Chevrolet'
    , 'Lonely Road'
    , 'Hang Tight Honey'
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , "Wind Up Missin' You"
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Beautiful As You'
    , "Think I'm In Love With You"
    , 'Love You, Miss You, Mean It'
    , 'Right About You' # Post Malone
    , "She's Somebody's Daughter (Reimagined)"
    , 'American Nights'
    , 'Am I Okay?'
    , 'Holy Smokes'
    , 'Young Love & Saturday Nights'
    , 'Love You Again'
    , 'Sweet Dreams'
    , 'Devil You Know'
    , 'Worst Way'
    , 'Relapse'
    , 'Wanna Be Loved'
]

In [8]:
%%sql df_song_labeled <<
SELECT
    ss.song_id
    , ss.song_name
    , art.artist_name
    , s2a.relationship_type
    , ss.untracked_weeks_on_chart
FROM song ss
LEFT JOIN song_to_artist s2a
    ON ss.song_id = s2a.song_id
LEFT JOIN artist art
    ON s2a.artist_id = art.artist_id
;

In [16]:
df_song_labeled[df_song_labeled['song_name'] == 'Wanna Be Loved']

,song_id,song_name,artist_name,relationship_type,untracked_weeks_on_chart
26,45,Wanna Be Loved,The Red Clay Strays,Lead artist,0.0


In [11]:
existing_song_to_rank = {}
new_song_to_rank = {}
for song, rank_ix in zip(rankings, range(1,51)):
    print(df_song_labeled.loc[df_song_labeled['song_name'] == song].to_markdown())
    song_id = input("Please enter the song_id or 'New Song'\n")
    if song_id.lower() == 'new song':
        new_song_to_rank[song] = rank_ix
    else:
        existing_song_to_rank[int(song_id)] = rank_ix

|    |   song_id | song_name          | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-------------------|:--------------|:--------------------|---------------------------:|
| 16 |        34 | A Bar Song (Tipsy) | Shaboozey     | Lead artist         |                          0 |
Please enter the song_id or 'New Song'
34
|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 20 |        38 | I Had Some Help | Morgan Wallen | Other main artist   |                          0 |
| 62 |        38 | I Had Some Help | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'
38
|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|-

| song_id   | song_name   | artist_name   | relationship_type   | untracked_weeks_on_chart   |
|-----------|-------------|---------------|---------------------|----------------------------|
Please enter the song_id or 'New Song'
New song
|    |   song_id | song_name                 | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:--------------------------|:--------------|:--------------------|---------------------------:|
| 33 |        53 | You Look Like You Love Me | Riley Green   | Featured artist     |                          0 |
| 65 |        53 | You Look Like You Love Me | Ella Langley  | Lead artist         |                          0 |
Please enter the song_id or 'New Song'
53
| song_id   | song_name   | artist_name   | relationship_type   | untracked_weeks_on_chart   |
|-----------|-------------|---------------|---------------------|----------------------------|
Please enter the song_id or 'New Song'
New Song
| song_id   | song_name   

Please enter the song_id or 'New Song'
49


In [18]:
print(json.dumps(existing_song_to_rank, indent = 2))
print(json.dumps(new_song_to_rank, indent = 2))

{
  "34": 1,
  "38": 2,
  "52": 3,
  "32": 4,
  "40": 5,
  "43": 6,
  "64": 7,
  "58": 8,
  "37": 9,
  "27": 10,
  "48": 13,
  "57": 15,
  "47": 19,
  "21": 20,
  "24": 23,
  "50": 24,
  "53": 26,
  "25": 30,
  "65": 31,
  "42": 32,
  "30": 34,
  "39": 36,
  "22": 37,
  "33": 38,
  "44": 40,
  "56": 41,
  "62": 42,
  "26": 43,
  "29": 44,
  "23": 45,
  "41": 46,
  "36": 47,
  "35": 48,
  "49": 49,
  "45": 50
}
{
  "Wrong Ones": 11,
  "Losers": 12,
  "California Sober": 14,
  "What Don't Belong To Me": 16,
  "Finer Things": 17,
  "Nosedive": 18,
  "Yours": 21,
  "Have The Heart": 22,
  "Goes Without Saying": 25,
  "Missin' You Like This": 27,
  "Hide My Gun": 28,
  "Devil I've Been": 29,
  "Never Love You Again": 33,
  "M-E-X-I-C-O": 35,
  "Right About You": 39
}


## Add artists on any new songs that aren't in the artist table yet to its CSV

    , 'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'California Sober' # Post Malone feat Chris Stapleton
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'Yours' # Post Malone
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Right About You' # Post Malone

In [65]:
df_artist = pd.read_csv('source_data/artist.csv')

In [66]:
new_song_artists = [
    'Post Malone' # 15
    , 'Tim McGraw' # new
    , 'Jelly Roll' # 5
    , 'Chris Stapleton' # 12
    , 'Hank Williams' # new
    , 'Lainey Wilson' # 11
    , 'Brad Paisley' # new
    , 'Luke Combs' # 6
    , 'HARDY' # new
    , 'ERNEST' # 37
    , 'Sierra Ferrell' # new
    , 'Billy Strings' # new
]
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
4,5,Jelly Roll,0.0,2024-08-23
5,6,Luke Combs,0.0,2024-08-23
10,11,Lainey Wilson,1.0,2024-08-23
11,12,Chris Stapleton,0.0,2024-08-23
14,15,Post Malone,0.0,2024-08-23
36,37,ERNEST,0.0,2024-08-23


In [62]:
df_artist['artist_id'].max()

54

In [67]:
df_artist = pd.concat(
    [
        df_artist
        , pd.DataFrame(
            np.array([
                [55, 'Tim McGraw', 0, current_date]
                , [56, 'Hank Williams', 0, current_date]
                , [57, 'Brad Paisley', 0, current_date]
                , [58, 'HARDY', 0, current_date]
                , [59, 'Sierra Ferrell', 0, current_date]
                , [60, 'Billy Strings', 0, current_date]
            ])
            , columns = ['artist_id', 'artist_name', 'pct_female', 'last_update']
        )
    ]
    , ignore_index = True
)

In [71]:
df_artist.to_csv('source_data/artist.csv', index=False)

## Add all new songs to the song table CSV

In [124]:
df_song_raw = pd.read_csv('source_data/song.csv')

In [125]:
max_song_id = df_song_raw['song_id'].max()
array_new_songs = []
for song_name in [
    'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'California Sober' # Post Malone feat Chris Stapleton
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'Yours' # Post Malone
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Right About You' # Post Malone
]:
    max_song_id += 1
    array_new_songs += [[max_song_id, song_name, 0, current_date]]

array_new_songs

[[90, 'Wrong Ones', 0, '2024-08-29'],
 [91, 'Losers', 0, '2024-08-29'],
 [92, 'California Sober', 0, '2024-08-29'],
 [93, "What Don't Belong To Me", 0, '2024-08-29'],
 [94, 'Finer Things', 0, '2024-08-29'],
 [95, 'Nosedive', 0, '2024-08-29'],
 [96, 'Yours', 0, '2024-08-29'],
 [97, 'Goes Without Saying', 0, '2024-08-29'],
 [98, "Missin' You Like This", 0, '2024-08-29'],
 [99, 'Hide My Gun', 0, '2024-08-29'],
 [100, "Devil I've Been", 0, '2024-08-29'],
 [101, 'Never Love You Again', 0, '2024-08-29'],
 [102, 'M-E-X-I-C-O', 0, '2024-08-29'],
 [103, 'Right About You', 0, '2024-08-29']]

In [82]:
df_song_raw = pd.concat(
    [
        df_song_raw
        , pd.DataFrame(
            np.array(array_new_songs)
            , columns = ['song_id', 'song_name', 'untracked_weeks_on_chart', 'last_update']
        )
    ]
    , ignore_index = True
)

In [126]:
# Update untracked_weeks_on_chart if needed
df_song_raw[-len(array_new_songs):]

,song_id,song_name,untracked_weeks_on_chart,last_update
75,76,Wrong Ones,0.0,2024-08-29
76,77,Losers,0.0,2024-08-29
77,78,California Sober,0.0,2024-08-29
78,79,What Don't Belong To Me,0.0,2024-08-29
79,80,Finer Things,0.0,2024-08-29
80,81,Nosedive,0.0,2024-08-29
81,82,Yours,0.0,2024-08-29
82,83,Goes Without Saying,0.0,2024-08-29
83,84,Missin' You Like This,0.0,2024-08-29
84,85,Hide My Gun,0.0,2024-08-29


In [127]:
df_song_raw.to_csv('source_data/song.csv', index=False)

## Add all new songs to the song_to_artist table CSV

In [95]:
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
4,5,Jelly Roll,0.0,2024-08-23
5,6,Luke Combs,0.0,2024-08-23
10,11,Lainey Wilson,1.0,2024-08-23
11,12,Chris Stapleton,0.0,2024-08-23
14,15,Post Malone,0.0,2024-08-23
36,37,ERNEST,0.0,2024-08-23
54,55,Tim McGraw,0,2024-08-29
55,56,Hank Williams,0,2024-08-29
56,57,Brad Paisley,0,2024-08-29
57,58,HARDY,0,2024-08-29


In [97]:
df_s2a_raw = pd.read_csv('source_data/song_to_artist.csv')

In [128]:
array_new_s2a = []
for s2a in [ # song_id, artist_id, relationship_conjunction, relationship_index
    [76, 15, '', '']
    , [76, 55, '', 1]
    , [77, 15, '', '']
    , [77, 5, '', 1]
    , [78, 15, '', '']
    , [78, 12, '', 1]
    , [79, 15, '', '']
    , [80, 15, '', '']
    , [80, 56, '', 1]
    , [81, 15, '', '']
    , [81, 11, '', 1]
    , [82, 15, '', '']
    , [83, 15, '', '']
    , [83, 57, '', 1]
    , [84, 15, '', '']
    , [84, 6, '', 1]
    , [85, 15, '', '']
    , [85, 58, '', 1]
    , [86, 15, '', '']
    , [86, 37, '', 1]
    , [87, 15, '', '']
    , [87, 59, '', 1]
    , [88, 15, '', '']
    , [88, 60, '', 1]
    , [89, 15, '', '']
]:
    if s2a[3] == '': # Only lead artists don't have a relationship index
        reln = 'Lead artist'
    elif s2a[2] == '': # Featured artists don't have an explicit relationship conjunction
        reln = 'Featured artist' 
    else:
        reln = 'Other main artist'
        
    array_new_s2a += [[
        '-'.join([str(s2a[0]), reln, str(s2a[3])])
        , s2a[0]
        , s2a[1]
        , reln
        , s2a[2]
        , s2a[3]
        , current_date
    ]]
    
# array_new_s2a

In [120]:
df_s2a_raw = pd.concat(
    [
        df_s2a_raw
        , pd.DataFrame(
            np.array(array_new_s2a)
            , columns = [
                'song_to_artist_pk'
                , 'song_id'
                , 'artist_id'
                , 'relationship_type'
                , 'relationship_conjunction'
                , 'relationship_index'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
).fillna('')

In [129]:
df_s2a_raw.tail(1)

,song_to_artist_pk,song_id,artist_id,relationship_type,relationship_conjunction,relationship_index,last_update
127,89-Lead artist-,89,15,Lead artist,,,2024-08-29


In [130]:
df_s2a_raw.to_csv('source_data/song_to_artist.csv', index=False)

In [124]:
## Convert new_song_to_rank to the id_to_rankix format of existing_song_to_rank

In [125]:
## Stack the two dicts of songs and insert to CSV with more info like current_date for last_update